In [1]:
cd /home/users/cdonoso/astromer/ASTROMER/

/home/users/cdonoso/astromer/ASTROMER


In [49]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import json
import os

from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [36]:
from core.encoder import Encoder
from core.astromer import get_ASTROMER
from core.data import pretraining_records

In [37]:
datapath = './data/records/alcock/fold_0/alcock_100/'
num_cls = pd.read_csv(os.path.join(datapath, 'objects.csv')).shape[0]

train_batches = pretraining_records(os.path.join(datapath, 'train'),
                                    256, max_obs=200,
                                    msk_frac=0., rnd_frac=0., same_frac=0.,
                                    sampling=False, shuffle=True,
                                    n_classes=num_cls)

val_batches = pretraining_records(os.path.join(datapath, 'train'),
                                  256, max_obs=200,
                                  msk_frac=0., rnd_frac=0., same_frac=0.,
                                  sampling=False, shuffle=False,
                                  n_classes=num_cls)

[INFO] Processing labels
[INFO] Processing labels


In [52]:
conf_file = os.path.join('./weights/astromer_10022021/finetuning/alcock_f1/', 'conf.json')
with open(conf_file, 'r') as handle:
    conf = json.load(handle)
    
model = get_ASTROMER(num_layers=conf['layers'],
                     d_model   =conf['head_dim'],
                     num_heads =conf['heads'],
                     dff       =conf['dff'],
                     base      =conf['base'],
                     dropout   =conf['dropout'],
                     maxlen    =conf['max_obs'],
                     use_leak  =conf['use_leak'])

weights_path = '{}/weights'.format('./weights/astromer_10022021/finetuning/alcock_f1/')
model.load_weights(weights_path)
model.trainable=False

In [53]:
encoder = model.get_layer('encoder')

In [58]:
serie  = Input(shape=(200, 1),
              batch_size=None,
              name='input')
times  = Input(shape=(200, 1),
              batch_size=None,
              name='times')
mask   = Input(shape=(200, 1),
              batch_size=None,
              name='mask')

placeholder = {'input':serie,
               'mask_in':mask,
               'times':times}

mask_in = 1.-placeholder['mask_in']
x = encoder(placeholder, training=False)
x = x * mask_in
x = tf.reduce_sum(x, 1)/tf.reduce_sum(mask_in, 1)

x = Dense(1024, name='FCN1')(x)
x = Dense(512, name='FCN2')(x)
x = Dense(256, name='FCN3')(x)
x = Dense(num_cls, name='FCN4')(x)
    
clf = Model(inputs=placeholder, outputs=x, name="MLPCLF")

In [59]:
clf.summary()

Model: "MLPCLF"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 200, 1)]     0                                            
__________________________________________________________________________________________________
mask (InputLayer)               [(None, 200, 1)]     0                                            
__________________________________________________________________________________________________
times (InputLayer)              [(None, 200, 1)]     0                                            
__________________________________________________________________________________________________
encoder (Encoder)               (None, None, 256)    660736      input[0][0]                      
                                                                 mask[0][0]                  

In [60]:
clf.compile(optimizer=Adam(learning_rate=1e-3),
              loss=CategoricalCrossentropy(from_logits=True),
              metrics='accuracy')

In [61]:
estop = EarlyStopping(monitor='val_loss',
                      min_delta=0,
                      patience=30,
                      verbose=0,
                      mode='auto',
                      baseline=None,
                      restore_best_weights=True)
    
_ = clf.fit(train_batches,
            epochs=1000,
            callbacks=[estop],
            validation_data=val_batches)

Epoch 1/1000
6/6 [==============================] - 3s 288ms/step - loss: 16.9625 - accuracy: 0.1807 - val_loss: 9.9815 - val_accuracy: 0.1896
Epoch 2/1000
6/6 [==============================] - 1s 196ms/step - loss: 6.8471 - accuracy: 0.1472 - val_loss: 5.2183 - val_accuracy: 0.1821
Epoch 3/1000
6/6 [==============================] - 1s 197ms/step - loss: 3.2761 - accuracy: 0.1848 - val_loss: 2.6464 - val_accuracy: 0.2074
Epoch 4/1000
6/6 [==============================] - 1s 198ms/step - loss: 2.7035 - accuracy: 0.2005 - val_loss: 1.7725 - val_accuracy: 0.1786
Epoch 5/1000
6/6 [==============================] - 1s 196ms/step - loss: 2.0199 - accuracy: 0.2081 - val_loss: 1.8606 - val_accuracy: 0.2820
Epoch 6/1000
6/6 [==============================] - 1s 198ms/step - loss: 1.7669 - accuracy: 0.2101 - val_loss: 1.6948 - val_accuracy: 0.2601
Epoch 7/1000
6/6 [==============================] - 1s 200ms/step - loss: 1.7147 - accuracy: 0.2389 - val_loss: 1.6360 - val_accuracy: 0.2574
Epoch

6/6 [==============================] - 1s 199ms/step - loss: 1.0473 - accuracy: 0.5770 - val_loss: 0.9887 - val_accuracy: 0.6105
Epoch 59/1000
6/6 [==============================] - 1s 197ms/step - loss: 1.0209 - accuracy: 0.5873 - val_loss: 0.9788 - val_accuracy: 0.6215
Epoch 60/1000
6/6 [==============================] - 1s 199ms/step - loss: 1.0284 - accuracy: 0.6010 - val_loss: 0.9832 - val_accuracy: 0.6057
Epoch 61/1000
6/6 [==============================] - 1s 194ms/step - loss: 1.0393 - accuracy: 0.5873 - val_loss: 0.9738 - val_accuracy: 0.6126
Epoch 62/1000
6/6 [==============================] - 1s 196ms/step - loss: 1.0269 - accuracy: 0.5975 - val_loss: 0.9708 - val_accuracy: 0.6119
Epoch 63/1000
6/6 [==============================] - 1s 198ms/step - loss: 1.0400 - accuracy: 0.5907 - val_loss: 0.9748 - val_accuracy: 0.6057
Epoch 64/1000
6/6 [==============================] - 1s 196ms/step - loss: 1.0499 - accuracy: 0.5832 - val_loss: 0.9741 - val_accuracy: 0.5975
Epoch 65/1000

6/6 [==============================] - 1s 197ms/step - loss: 0.9625 - accuracy: 0.6010 - val_loss: 0.9150 - val_accuracy: 0.6318
Epoch 116/1000
6/6 [==============================] - 1s 198ms/step - loss: 0.9641 - accuracy: 0.6201 - val_loss: 0.9118 - val_accuracy: 0.6461
Epoch 117/1000
6/6 [==============================] - 1s 198ms/step - loss: 0.9754 - accuracy: 0.6270 - val_loss: 0.9092 - val_accuracy: 0.6427
Epoch 118/1000
6/6 [==============================] - 1s 197ms/step - loss: 0.9216 - accuracy: 0.6578 - val_loss: 0.9029 - val_accuracy: 0.6667
Epoch 119/1000
6/6 [==============================] - 1s 197ms/step - loss: 0.9052 - accuracy: 0.6468 - val_loss: 0.8855 - val_accuracy: 0.6626
Epoch 120/1000
6/6 [==============================] - 1s 195ms/step - loss: 0.9293 - accuracy: 0.6331 - val_loss: 0.8984 - val_accuracy: 0.6407
Epoch 121/1000
6/6 [==============================] - 1s 194ms/step - loss: 0.9449 - accuracy: 0.6126 - val_loss: 0.9019 - val_accuracy: 0.6489
Epoch 1

Epoch 172/1000
6/6 [==============================] - 1s 202ms/step - loss: 0.8881 - accuracy: 0.6619 - val_loss: 0.8469 - val_accuracy: 0.6838
Epoch 173/1000
6/6 [==============================] - 1s 200ms/step - loss: 0.8895 - accuracy: 0.6585 - val_loss: 0.8489 - val_accuracy: 0.6790
Epoch 174/1000
6/6 [==============================] - 1s 198ms/step - loss: 0.8916 - accuracy: 0.6400 - val_loss: 0.8490 - val_accuracy: 0.6762
Epoch 175/1000
6/6 [==============================] - 1s 196ms/step - loss: 0.9129 - accuracy: 0.6201 - val_loss: 0.8631 - val_accuracy: 0.6626
Epoch 176/1000
6/6 [==============================] - 1s 198ms/step - loss: 0.9357 - accuracy: 0.6407 - val_loss: 0.8968 - val_accuracy: 0.6516
Epoch 177/1000
6/6 [==============================] - 1s 194ms/step - loss: 0.9733 - accuracy: 0.6229 - val_loss: 0.8726 - val_accuracy: 0.6838
Epoch 178/1000
6/6 [==============================] - 1s 195ms/step - loss: 0.8694 - accuracy: 0.6858 - val_loss: 0.8520 - val_accuracy:

Epoch 229/1000
6/6 [==============================] - 1s 195ms/step - loss: 0.8932 - accuracy: 0.6468 - val_loss: 0.8517 - val_accuracy: 0.6988
Epoch 230/1000
6/6 [==============================] - 1s 197ms/step - loss: 0.8689 - accuracy: 0.6749 - val_loss: 0.8494 - val_accuracy: 0.6988
Epoch 231/1000
6/6 [==============================] - 1s 196ms/step - loss: 0.8707 - accuracy: 0.6571 - val_loss: 0.8459 - val_accuracy: 0.6831
Epoch 232/1000
6/6 [==============================] - 1s 199ms/step - loss: 0.8779 - accuracy: 0.6660 - val_loss: 0.8487 - val_accuracy: 0.6824
Epoch 233/1000
6/6 [==============================] - 1s 200ms/step - loss: 0.8710 - accuracy: 0.6667 - val_loss: 0.8342 - val_accuracy: 0.6954
Epoch 234/1000
6/6 [==============================] - 1s 200ms/step - loss: 0.8767 - accuracy: 0.6687 - val_loss: 0.8316 - val_accuracy: 0.6934
Epoch 235/1000
6/6 [==============================] - 1s 197ms/step - loss: 0.8856 - accuracy: 0.6653 - val_loss: 0.8327 - val_accuracy: